**Load Library**

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq  # ✅ Perbaikan import
import os

# ✅ Ganti dengan API key Groq milikmu
os.environ["GROQ_API_KEY"] = "YOUR_API_KEY_HERE"

**Load File**

In [19]:
loader = UnstructuredFileLoader("proklamasi.txt")
documents = loader.load()


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


**Split Document**

In [20]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)

docs = text_splitter.split_documents(documents)

Created a chunk of size 3224, which is longer than the specified 500
Created a chunk of size 4663, which is longer than the specified 500
Created a chunk of size 1469, which is longer than the specified 500
Created a chunk of size 3417, which is longer than the specified 500
Created a chunk of size 2087, which is longer than the specified 500
Created a chunk of size 811, which is longer than the specified 500
Created a chunk of size 1399, which is longer than the specified 500
Created a chunk of size 1208, which is longer than the specified 500
Created a chunk of size 1645, which is longer than the specified 500
Created a chunk of size 1040, which is longer than the specified 500
Created a chunk of size 874, which is longer than the specified 500
Created a chunk of size 1591, which is longer than the specified 500
Created a chunk of size 606, which is longer than the specified 500


**Embeddings dan Vectorstore**

In [21]:
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

C:\Users\Acer\AppData\Local\Temp\ipykernel_17496\2354805888.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


**Retriever**

In [22]:
retriever = vectorstore.as_retriever()


**Load Model**

In [23]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")


**QnA**

In [24]:
qa_chain = load_qa_chain(llm, chain_type="stuff")
query = "Apa isi dari teks proklamasi kemerdekaan Indonesia?"
docs_relevant = retriever.get_relevant_documents(query)
response = qa_chain.run(input_documents=docs_relevant, question=query)
print(response)


Isi teks Proklamasi Kemerdekaan Indonesia adalah:

"Kami bangsa Indonesia dengan ini menyatakan kemerdekaan Indonesia. Hal-hal yang mengenai pemindahan kekuasaan dan lain-lain diselenggarakan dengan cara seksama dan dalam tempo yang sesingkat-singkatnya. Jakarta, 17 - 8 - '05 Wakil2 bangsa Indonesia, Soekarno – Hatta"


In [28]:
qa_chain = load_qa_chain(llm, chain_type="stuff")

questions = [
    "Apa isi dari teks proklamasi kemerdekaan Indonesia?",
    "Siapa yang menandatangani teks proklamasi?",
    "Kapan proklamasi kemerdekaan Indonesia dibacakan?",
    "Mengapa teks proklamasi menggunakan ejaan lama?"
]

for query in questions:
    docs_relevant = retriever.get_relevant_documents(query)
    response = qa_chain.run(input_documents=docs_relevant, question=query)
    print(f"Pertanyaan: {query}")
    print(f"Jawaban: {response}")
    print("-" * 50)

Pertanyaan: Apa isi dari teks proklamasi kemerdekaan Indonesia?
Jawaban: Isi teks Proklamasi Kemerdekaan Indonesia adalah:

"Kami bangsa Indonesia dengan ini menyatakan kemerdekaan Indonesia. Hal-hal yang mengenai pemindahan kekuasaan dan lain-lain diselenggarakan dengan cara seksama dan dalam tempo yang sesingkat-singkatnya. Jakarta, 17 - 8 - '05 Wakil2 bangsa Indonesia, Soekarno – Hatta"
--------------------------------------------------
Pertanyaan: Siapa yang menandatangani teks proklamasi?
Jawaban: Menurut teks, yang menandatangani teks Proklamasi Otentik adalah Ir. Soekarno dan Drs. Mohammad Hatta.
--------------------------------------------------
Pertanyaan: Kapan proklamasi kemerdekaan Indonesia dibacakan?
Jawaban: Proklamasi kemerdekaan Indonesia dibacakan pada tanggal 17 Agustus 1945.
--------------------------------------------------
Pertanyaan: Mengapa teks proklamasi menggunakan ejaan lama?
Jawaban: Teks proklamasi tidak menggunakan ejaan lama. Teks proklamasi yang asli, y

**Save Model**

In [ ]:
import pickle

with open("retriever.pkl", "wb") as f:
    pickle.dump(retriever, f)

**Load Model**

In [ ]:
with open("retriever.pkl", "rb") as f:
    retriever = pickle.load(f)

Analisis
Proyek ini sudah berhasil menjalankan sistem RAG sederhana dengan memanfaatkan LangChain, FAISS, dan LLM dari Groq. Sistem bisa mengambil informasi dari dokumen dan menjawab pertanyaan dengan cukup baik. Namun, masih ada keterbatasan, seperti proses input data yang masih manual dan tampilan yang hanya bisa digunakan lewat notebook.

Rekomendasi
Agar lebih mudah digunakan, disarankan membuat tampilan antarmuka seperti web sederhana. Selain itu, sebaiknya sistem bisa memperbarui data secara otomatis tanpa upload manual.

Jawaban Pertanyaan
1. Mengapa menggunakan LLM tersebut?
LLM seperti llama3-8b-8192 digunakan karena memiliki kemampuan memahami konteks dalam teks panjang dengan baik, efisien secara komputasi, dan cocok untuk kebutuhan retrieval-augmented generation (RAG) berbahasa Indonesia. Model ini juga mudah diakses melalui Groq yang menawarkan kecepatan inferensi tinggi.

2. Mengapa memilih model embedding tersebut?
Model embedding dari sentence-transformers (HuggingFaceEmbeddings) dipilih karena telah terlatih untuk merepresentasikan semantik teks secara akurat ke dalam bentuk vektor, mendukung bahasa Indonesia, serta kompatibel dengan berbagai framework retrieval seperti FAISS.

3. Mengapa memilih vector DB tersebut?
FAISS dipilih sebagai vector database karena bersifat open-source, ringan, cepat dalam pencarian vektor mirip (nearest neighbor), dan mudah diintegrasikan dengan pipeline NLP seperti LangChain. Cocok untuk skenario aplikasi RAG berskala kecil hingga menengah.

4. Bagaimana melakukan update knowledge base?
Knowledge base dapat diperbarui dengan menambahkan dokumen baru ke dalam vector store, menghitung embedding-nya menggunakan model yang sama, lalu menggabungkannya dengan data sebelumnya di FAISS. Pembaruan ini tidak perlu melakukan retraining ulang seluruh sistem.

5. Apa saja kekurangan dari aplikasi RAG yang dikembangkan?
Beberapa kekurangannya meliputi keterbatasan konteks jika jumlah token terlalu besar, hasil jawaban sangat tergantung pada kualitas dan relevansi dokumen yang terindeks, serta tidak mendukung reasoning kompleks jika data sumber tidak lengkap. Selain itu, pembaruan skala besar dapat memerlukan reindexing ulang.